In [ ]:
#| hide
from lightrag1.core import *
#| default_exp cb2



In [4]:
#| export
import os
import json

def load_or_compute_embeddings(compute_embeddings):
    """
    Checks for required precomputed files and either loads them or computes and saves them.
    
    Parameters:
      compute_embeddings (callable): A function that computes and returns the embeddings.
    
    Returns:
      dict: The loaded or newly computed embeddings.
    """
    required_files = [
        "graph_chunk_entity_relation.graphml",
        "kv_store_text_chunks.json",
        "kv_store_doc_status.json",
        "vdb_chunks.json",
        "kv_store_full_docs.json",
        "vdb_entities.json",
        "kv_store_llm_response_cache.json",
        "vdb_relationships.json"
    ]
    
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print("Missing files detected:", missing_files)
        # Compute embeddings using the provided function
        embeddings = compute_embeddings()
        # Save results to one or more of the required files
        # (You may need to adapt this to your file structure)
        with open("vdb_chunks.json", "w", encoding="utf-8") as f:
            json.dump(embeddings, f)
        # Save other files as needed...
    else:
        print("All precomputed files found. Loading cached embeddings...")
        with open("vdb_chunks.json", "r", encoding="utf-8") as f:
            embeddings = json.load(f)
    
    return embeddings


In [10]:

#| export
import asyncio
import nest_asyncio
import os
import inspect
import logging
import csv

from lightrag import LightRAG, QueryParam
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.utils import EmbeddingFunc
from lightrag.kg.shared_storage import initialize_pipeline_status

nest_asyncio.apply()

WORKING_DIR = "./dickens"

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)


async def initialize_rag():
    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=ollama_model_complete,
        llm_model_name="deepseek-r1:1.5b1a",
        llm_model_max_async=4,
        llm_model_max_token_size=32768,
        llm_model_kwargs={
            "host": "http://localhost:11434",
            "options": {"num_ctx": 32768},
        },
        embedding_func=EmbeddingFunc(
            embedding_dim=768,
            max_token_size=8192,
            func=lambda texts: ollama_embed(
                texts, embed_model="nomic-embed-text", host="http://localhost:11434"
            ),
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()

    return rag


async def print_stream(stream):
    async for chunk in stream:
        print(chunk, end="", flush=True)


def main():
    # Initialize RAG instance
    rag = asyncio.run(initialize_rag())

    # Read CSV data and build a formatted string
    csv_data = ""
    with open("./42a.csv", "r", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            csv_data += f"Topic: {row['Topic']}\n"
            csv_data += f"Key Concepts/Themes: {row['Key Concepts/Themes']}\n\n"

    # Insert the CSV data into the RAG system
    rag.insert(csv_data)

    # Test different query modes with updated questions

    print("\nNaive Search:")
    print(
        rag.query(
            "What are the main topics and their associated themes?",
            param=QueryParam(mode="naive")
        )
    )
    
    print("\nLocal Search:")
    print(
        rag.query(
            "Which topic emphasizes the discipline of sensory focus and aesthetic enhancement?",
            param=QueryParam(mode="local")
        )
    )

    print("\nGlobal Search:")
    print(
        rag.query(
            "How do the themes reflect ideas of personal growth and moderation across different topics?",
            param=QueryParam(mode="global")
        )
    )

    print("\nHybrid Search:")
    print(
        rag.query(
            "Can you summarize how each topic combines concepts of self-expression, beauty, and resource management?",
            param=QueryParam(mode="hybrid")
        )
    )

    # Stream response for one of the queries
    resp = rag.query(
        "Can you summarize how each topic combines concepts of self-expression, beauty, and resource management?",
        param=QueryParam(mode="hybrid", stream=True),
    )

    if inspect.isasyncgen(resp):
        asyncio.run(print_stream(resp))
    else:
        print(resp)


if __name__ == "__main__":
    main()


INFO: Process 72468 Shared-Data already initialized (multiprocess=False)
INFO:Load (5, 768) data
INFO:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_entities.json'} 5 data
INFO:Load (5, 768) data
INFO:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_relationships.json'} 5 data
INFO:Load (4, 768) data
INFO:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_chunks.json'} 4 data
INFO: Process 72468 storage namespace already initialized: [full_docs]
INFO: Process 72468 storage namespace already initialized: [text_chunks]
INFO: Process 72468 storage namespace already initialized: [llm_response_cache]
INFO: Process 72468 storage namespace already initialized: [doc_status]
INFO: Process 72468 storage namespace already initialized: [full_docs]
INFO: Process 72468 storage namespace already initialized: [text_chunks]
INFO: Process 72468 storage namespace already initialized: [llm_response_cache]
INFO: Proce


Naive Search:
<think>
Alright, I'm looking at this document chunk about different topics related to existential philosophy. The user has given me a bunch of chunks from various documents and is asking for an analysis on the main topics and their associated themes.

First, I'll need to identify the main topics listed. From what's provided, they're split into 30 sections each discussing a specific aspect like ethics, spirituality, etc. Each of these sections seems to be focused on different areas within existential philosophy.

Next, I should determine what the key themes are across all these topics. It looks like several major areas are covered: existential void, inner fulfillment, universal struggles, self-reflection, and so on. The document covers a wide range from personal growth and existence in space and time to ethical dilemmas and spiritual practices.

I need to make sure I'm not missing any important themes that tie these topics together. It seems like the common thread is the 

In [6]:
import asyncio

# Reinitialize the RAG instance; this will load data from the existing working directory "./dickens"
rag = asyncio.run(initialize_rag())

# Example: perform a query to extract data from the Dickens working directory
query_text = "Please extract and summarize all topics and themes stored in the Dickens dataset."
result = rag.query(query_text, param=QueryParam(mode="global"))

# If the result is a streaming response, consume it asynchronously; otherwise, print the result directly.
if hasattr(result, '__aiter__'):
    async def consume_stream():
        full_response = ""
        async for chunk in result:
            full_response += chunk
        print(full_response)
    asyncio.run(consume_stream())
else:
    print(result)


INFO: Process 72468 Shared-Data already initialized (multiprocess=False)
INFO:Load (5, 768) data
INFO:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_entities.json'} 5 data
INFO:Load (5, 768) data
INFO:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_relationships.json'} 5 data
INFO:Load (4, 768) data
INFO:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_chunks.json'} 4 data
INFO: Process 72468 storage namespace already initialized: [full_docs]
INFO: Process 72468 storage namespace already initialized: [text_chunks]
INFO: Process 72468 storage namespace already initialized: [llm_response_cache]
INFO: Process 72468 storage namespace already initialized: [doc_status]
INFO: Process 72468 storage namespace already initialized: [full_docs]
INFO: Process 72468 storage namespace already initialized: [text_chunks]
INFO: Process 72468 storage namespace already initialized: [llm_response_cache]
INFO: Proce

<think>
Alright, I'm trying to figure out how to approach this task. So, from what I understand, the user has given me a knowledge base that includes entities like people and technology, along with relationships between them and their timestamps. The goal is to extract and summarize all the topics and themes stored in this dataset.

First, I need to look through the provided knowledge base. Let's see:

The entities are:
- "id", "entity", "type", "description"
Examples: Jordan, Taylor, Alex, The Device, Cruz

The relationships include:
- source, target, description, keywords, weight, rank
Each relationship seems to be a connection between two people or entities with some sort of interaction. For example, Jordan is interacting with Taylor regarding the device.

Looking at the sources, there are 10 topics/locales listed: 10 Topics / Locales. Each topic has its own key concepts and themes. These seem to cover various aspects like pragmatism in absolutes too much variability, complexity in 

In [7]:
#| export
# Assuming the `rag` instance has already been created in a previous cell

print("\nAdditional Questions:")

# Naive mode query: exploring economic aspects
print("Naive Query:")
print(
    rag.query(
        "What topic discusses economic prosperity and its impact on community life?",
        param=QueryParam(mode="naive")
    )
)

# Local mode query: connecting artistic expression with systematic resource allocation
print("\nLocal Query:")
print(
    rag.query(
        "Which topic integrates artistic expression with systematic resource allocation?",
        param=QueryParam(mode="local")
    )
)

# Global mode query: comprehensive analysis of aesthetic themes
print("\nGlobal Query:")
print(
    rag.query(
        "Can you provide a comprehensive analysis of aesthetic themes across all topics?",
        param=QueryParam(mode="global")
    )
)



Additional Questions:
Naive Query:
<think>
Okay, let me try to figure this out. The user is asking about economic prosperity and its impact on community life. From the document chunks provided, I see a bunch of topics like existential void, meditation, and various forms of meditation. Oh wait, but those aren't directly related to economics or community life.

Hmm, so maybe the user wants an overview of how economic prosperity affects communities. Let me think about that. Economic prosperity generally refers to when a country has enough resources to provide basic needs, like food, shelter, healthcare, and education. It's about having enough people for all kinds of activities and maintaining social stability.

Community life changes when there's prosperity because everyone contributes more to the society. For example, in a prosperous community, everyone gets enough support and opportunities. There might be better access to resources, so residents can live independently more easily. Also

In [8]:
#| export
import time

def print_query_metrics(query_text, mode):
    start_time = time.time()
    result = rag.query(query_text, param=QueryParam(mode=mode))
    duration = time.time() - start_time
    # Check if the result is an async generator; if so, run the stream synchronously for metrics.
    if hasattr(result, '__aiter__'):
        async def get_full_response():
            response = ""
            async for chunk in result:
                response += chunk
            return response
        result = asyncio.run(get_full_response())
    print(f"Query: {query_text}")
    print(f"Mode: {mode}")
    print("Response:", result)
    print(f"Time taken: {duration:.2f} seconds\n")

print("\nMetrics for Additional Queries:")

# Example metric measurement for a naive mode query
print_query_metrics(
    "How does the system interpret the integration of self-expression with beauty?", 
    "naive"
)

# Example metric measurement for a global mode query
print_query_metrics(
    "Can you detail the thematic evolution in topics emphasizing art?", 
    "global"
)



Metrics for Additional Queries:
Query: How does the system interpret the integration of self-expression with beauty?
Mode: naive
Response: <think>
Alright, let me break this down step by step. The user is asking how the system interprets the integration of self-expression with beauty. I need to understand both "self-expression" and "beauty," then see how they're connected in the context provided.

First, looking at the document chunks, topics like Buddhism (self-expression) and artistic psychology (beauty) are discussed. The system probably looks for specific keywords or terms that indicate self-expression and beauty separately, then assesses how they might be linked.

I should consider if there's a common theme or metaphor where someone's self-expression is tied to their beauty or presence in some way. For example, in meditation, one might express inner thoughts as beauty. Or in painting, expressing emotions directly reflects self-expression of beauty.

The user might be curious abou

In [12]:
#| export
import time

def print_query_metrics(query_text, mode):
    start_time = time.time()
    result = rag.query(query_text, param=QueryParam(mode=mode))
    duration = time.time() - start_time
    # Check if the result is an async generator; if so, run the stream synchronously for metrics.
    if hasattr(result, '__aiter__'):
        async def get_full_response():
            response = ""
            async for chunk in result:
                response += chunk
            return response
        result = asyncio.run(get_full_response())
    print(f"Query: {query_text}")
    print(f"Mode: {mode}")
    print("Response:", result)
    print(f"Time taken: {duration:.2f} seconds\n")

print("\nMetrics for Additional Queries:")

# Example metric measurement for a naive mode query
print_query_metrics(
    "How could you summarize the system?", 
    "naive"
)

# Example metric measurement for a global mode query
print_query_metrics(
    "Can you list 82 keywords from the document chunks provided?", 
    "global"
)



Metrics for Additional Queries:
Query: How could you summarize the system?
Mode: naive
Response: <think>
Alright, I need to figure out how to respond to the user's query about the document chunks they've provided. The user is asking for a summary of the system mentioned in their context. Let me look at the document chunk again.

The document chunk talks about various systems and concepts, such as existential void, emotional dissatisfaction, time-related topics like temporal awareness, meditation techniques, and others. It seems to cover different areas of philosophy or psychology.

I should start by identifying the main themes across all the document chunks. The key concepts seem to be:

1. Existential Concerns: Everything has a void, existential dissatisfaction is hard to overcome.
2. Time Management: Understanding temporal aspects like how time passes and being mindful of it.
3. Self-Expression: The importance of expressing oneself fully in different contexts, from personal to socia

In [9]:
import asyncio
import os
import json

def get_file_count_and_size(directory, extensions):
    """Returns the count and total size of files in a directory with given extensions."""
    files = [f for f in os.listdir(directory) if f.endswith(tuple(extensions))]
    total_size = sum(os.path.getsize(os.path.join(directory, f)) for f in files)
    return len(files), total_size

async def show_system_metrics():
    WORKING_DIR = "./dickens"
    FILE_CATEGORIES = {
        "Graph Chunks": ["graph_chunk_entity_relation.graphml"],
        "KV Stores": [
            "kv_store_text_chunks.json", "kv_store_doc_status.json", "kv_store_full_docs.json",
            "kv_store_llm_response_cache.json"
        ],
        "Vector DBs": ["vdb_chunks.json", "vdb_entities.json", "vdb_relationships.json"]
    }
    
    try:
        print("Document Storage Metrics:")
        doc_count, doc_size = get_file_count_and_size(WORKING_DIR, [".json"])
        print(f"Total Documents: {doc_count}, Size: {doc_size / 1024:.2f} KB")
        
        # Categorized Storage Metrics
        for category, files in FILE_CATEGORIES.items():
            category_size = sum(
                os.path.getsize(os.path.join(WORKING_DIR, f)) for f in files if os.path.exists(os.path.join(WORKING_DIR, f))
            )
            print(f"\n{category}:")
            print(f"  Files: {len(files)}")
            print(f"  Total Size: {category_size / 1024:.2f} KB")

        # Knowledge Graph Metrics
        kg_path = os.path.join(WORKING_DIR, "knowledge_graph")
        if os.path.exists(kg_path):
            kg_files = os.listdir(kg_path)
            print("\nKnowledge Graph Metrics:")
            print(f"  Total Graph Files: {len(kg_files)}")
        else:
            print("\nKnowledge Graph directory not found.")

    except Exception as e:
        print(f"Error accessing metrics: {str(e)}")

    # Storage Directory Structure
    print("\nStorage Directory Structure:")
    for root, dirs, files in os.walk(WORKING_DIR):
        print(f"{root}:")
        print(f"  Directories: {len(dirs)}")
        print(f"  Files: {len(files)}")
        break  # Show only the root directory

asyncio.run(show_system_metrics())


Document Storage Metrics:
Total Documents: 7, Size: 359.96 KB

Graph Chunks:
  Files: 1
  Total Size: 3.96 KB

KV Stores:
  Files: 4
  Total Size: 300.16 KB

Vector DBs:
  Files: 3
  Total Size: 59.80 KB

Knowledge Graph directory not found.

Storage Directory Structure:
./dickens:
  Directories: 0
  Files: 8


In [ ]:
# Cell 2: Final Working Metrics Analysis
import asyncio
import os
import json

async def show_system_metrics():
    rag = await initialize_rag()
    
    print("Analyzing RAG Storage Metrics:\n")
    
    try:
        # Document status analysis
        doc_status_path = os.path.join(WORKING_DIR, "kv_store_doc_status.json")
        if os.path.exists(doc_status_path):
            with open(doc_status_path, "r") as f:
                doc_status = json.load(f)
                print(f"📄 Document Status:")
                print(f"- Total documents tracked: {len(doc_status)}")
                processed = sum(1 for v in doc_status.values() if v.get('processed'))
                print(f"- Processed documents: {processed}")
                print(f"- Processing failures: {sum(1 for v in doc_status.values() if v.get('failed'))}")

        # Chunk storage analysis
        chunk_path = os.path.join(WORKING_DIR, "vdb_chunks.json")
        if os.path.exists(chunk_path):
            with open(chunk_path, "r") as f:
                chunk_data = json.load(f)
                print(f"\n📄 Chunk Storage (vdb_chunks.json):")
                print(f"- Total chunk entries: {len(chunk_data.get('data', []))}")
                if chunk_data.get('data'):
                    avg_size = sum(len(entry.get('text', '')) for entry in chunk_data['data']) / len(chunk_data['data'])
                    print(f"- Average chunk size: {avg_size:.1f} characters")

        # Knowledge graph analysis
        entity_path = os.path.join(WORKING_DIR, "vdb_entities.json")
        rel_path = os.path.join(WORKING_DIR, "vdb_relationships.json")
        
        if os.path.exists(entity_path):
            with open(entity_path, "r") as f:
                entity_data = json.load(f)
                print(f"\n📄 Knowledge Graph Entities (vdb_entities.json):")
                print(f"- Entity records: {len(entity_data.get('data', []))}")
        
        if os.path.exists(rel_path):
            with open(rel_path, "r") as f:
                rel_data = json.load(f)
                print(f"\n📄 Knowledge Graph Relationships (vdb_relationships.json):")
                print(f"- Relationship records: {len(rel_data.get('data', []))}")

    except Exception as e:
        print(f"\n⚠️ Error analyzing metrics: {str(e)}")

    # File system analysis
    print("\n📂 Directory Contents:")
    for item in os.listdir(WORKING_DIR):
        full_path = os.path.join(WORKING_DIR, item)
        if os.path.isfile(full_path):
            size = os.path.getsize(full_path)
            print(f"- {item} ({size} bytes)")

asyncio.run(show_system_metrics())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
def verify_data_ingestion():
    # Check document count matches CSV entries
    with open("./42a.csv") as f:
        csv_rows = sum(1 for _ in csv.DictReader(f))  # Should be 42
        
    with open(os.path.join(WORKING_DIR, "kv_store_doc_status.json")) as f:
        doc_status = json.load(f)
    
    print(f"CSV Entries: {csv_rows}")
    print(f"Ingested Documents: {len(doc_status)}")
    print(f"Processing Success Rate: {sum(1 for v in doc_status.values() if v['processed'])/len(doc_status):.1%}")

In [ ]:
def analyze_content_quality():
    with open(os.path.join(WORKING_DIR, "vdb_chunks.json")) as f:
        chunks = json.load(f)['data']
    
    # Verify theme preservation
    theme_coverage = Counter()
    for chunk in chunks:
        if 'themes' in chunk['metadata']:
            for theme in chunk['metadata']['themes']:
                theme_coverage[theme] += 1
                
    print("\nTheme Coverage in Chunks:")
    for theme, count in theme_coverage.most_common(10):
        print(f"- {theme}: {count} mentions")

In [ ]:
def validate_knowledge_graph():
    with open(os.path.join(WORKING_DIR, "vdb_entities.json")) as f:
        entities = json.load(f)['data']
    
    with open(os.path.join(WORKING_DIR, "vdb_relationships.json")) as f:
        relationships = json.load(f)['data']
    
    print(f"\nEntity Types: {len({e['metadata']['type'] for e in entities})}")
    print(f"Avg Connections per Entity: {len(relationships)/len(entities):.1f}")

In [ ]:
def pipeline_completeness():
    required_files = [
        "vdb_chunks.json",
        "vdb_entities.json",
        "vdb_relationships.json",
        "kv_store_doc_status.json"
    ]
    
    missing = [f for f in required_files if not os.path.exists(f)]
    print("\nMissing Files:" if missing else "\nAll Pipeline Files Present")
    for f in missing:
        print(f"- {f}")

In [ ]:
async def test_query_effectiveness(rag):
    test_cases = [
        ("What Nietzsche concepts are criticized?", "global"),
        ("Explain meditation techniques mentioned", "local"),
        ("How is beauty maximized?", "hybrid")
    ]
    
    for question, mode in test_cases:
        print(f"\nTesting '{question}' ({mode}):")
        result = rag.query(question, param=QueryParam(mode=mode))
        print(f"- Top 3 Relevant Chunks:")
        for i, chunk in enumerate(result['chunks'][:3], 1):
            print(f"  {i}. {chunk['metadata']['topic'][:50]}...")